# [VGG]

*KU LeeDongGyu*

### Contents

1. Almost Original VGG19 & Data Import
2. For Size = 64,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
3. For Size = 48,
```
1) My VGG11 (Pretraining)
2) My VGG16 (Pretrained)
3) My VGG19 (Pretrained)
4) My VGG11 (Convergence Speed Comparison)
```
4. For Size = 48, No Early Stopping
```
1) Epoch = 50
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
2) Epoch = 100
      (1) My VGG11
      (2) My VGG16
      (3) My VGG19
```
5. Model Save & Import

### Install Packages

In [0]:
pip install opencv-python

### Module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [4]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [6]:
from lrn import LRN
from f1score import macro_f1score,weighted_f1score

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D,GlobalMaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_file, to_categorical

In [8]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

## 1. Almost Original VGG19 & Data Import
---

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)


y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size = 224
x_train = np.array(x_train).reshape([-1,48,48,1])

In [0]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_train_zoom.shape

(28698, 224, 224, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_valid_zoom.shape

(3589, 224, 224, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [0]:
x_test_zoom.shape

(3588, 224, 224, 1)

In [0]:
# VGG19 모형과 관련된 초기값이 저장되어 있는 링크. 접속시 바로 다운됨. FC층을 어떻게 처리할 건지에 따라 두가지 초기값들이 있음.
# 이 값은 ImageNet 데이터를 가지고 학습한 초기값임.
# 내가 쓸 모형은 이 초기값이 필요 없기 때문에, 내가 초기값을 뽑아내는 작업을 해야한다.

# 다시한번 얘기하지만, 아래의 두 변수는 내가 아래의 코드에서 쓸 변수는 아님. 나중의 혹시모를 상황을 대비해 남겨놓음.
WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

# -*- coding: utf-8 -*-
'''VGG19 model for Keras.
# Reference:
- [Very Deep Convolutional Networks for Large-Scale Image Recognition](https://arxiv.org/abs/1409.1556)
'''
('https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg19_weights_tf_dim_ordering_tf_kernels.h5')
"""Instantiates the VGG19 architecture.
 Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,
 for best performance you should set
 `image_data_format="channels_last"` in your Keras config
 at ~/.keras/keras.json.
 The model and the weights are compatible with both
 TensorFlow and Theano. The data format
 convention used by the model is the one
 specified in your Keras config file.

 # Arguments
     include_top: whether to include the 3 fully-connected
         layers at the top of the network.
     weights: one of `None` (random initialization)
         or "imagenet" (pre-training on ImageNet).
     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
         to use as image input for the model.
     input_shape: optional shape tuple, only to be specified
         if `include_top` is False (otherwise the input shape
         has to be `(224, 224, 3)` (with `channels_last` data format)
         or `(3, 224, 244)` (with `channels_first` data format).
         It should have exactly 3 inputs channels,
         and width and height should be no smaller than 48.
         E.g. `(200, 200, 3)` would be one valid value.
     pooling: Optional pooling mode for feature extraction
         when `include_top` is `False`.
         - `None` means that the output of the model will be
             the 4D tensor output of the
             last convolutional layer.
         - `avg` means that global average pooling
             will be applied to the output of the
             last convolutional layer, and thus
             the output of the model will be a 2D tensor.
         - `max` means that global max pooling will
             be applied.
     classes: optional number of classes to classify images
         into, only to be specified if `include_top` is True, and
         if no `weights` argument is specified.
 # Returns
     A Keras model instance.
 """


'Instantiates the VGG19 architecture.\n Optionally loads weights pre-trained on ImageNet. Note that when using TensorFlow,\n for best performance you should set\n `image_data_format="channels_last"` in your Keras config\n at ~/.keras/keras.json.\n The model and the weights are compatible with both\n TensorFlow and Theano. The data format\n convention used by the model is the one\n specified in your Keras config file.\n\n # Arguments\n     include_top: whether to include the 3 fully-connected\n         layers at the top of the network.\n     weights: one of `None` (random initialization)\n         or "imagenet" (pre-training on ImageNet).\n     input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)\n         to use as image input for the model.\n     input_shape: optional shape tuple, only to be specified\n         if `include_top` is False (otherwise the input shape\n         has to be `(224, 224, 3)` (with `channels_last` data format)\n         or `(3, 224, 244)` (with 

In [0]:
# VGG19를 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.

def VGG19(input_shape=(224, 224, 1), classes=7, include_top=True, pooling=None, weights=None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(4096, activation='relu', name='fc1')(x)
        x = Dense(4096, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    if weights == 'imagenet': # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH, cache_subdir='models', file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP, cache_subdir='models',
                                    file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path)  # 경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model

In [0]:
model = VGG19(input_shape=(224, 224, 3), classes=7)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

## 2. For Size = 64,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 논문에서 VGG19 의 FC층은 VGG11로 pretraining하여 얻은 초기치로 설정하였음.
# 기존의 VGG11을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
size = 64 # 적당한 크기로 잡음.
x_train = np.array(x_train).reshape([-1,48,48,1])

In [11]:
x_train.shape

(28698, 48, 48, 1)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,1],dtype="float32")
for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [13]:
x_train_zoom.shape

(28698, 64, 64, 1)

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,1])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,1],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [15]:
x_valid_zoom.shape

(3589, 64, 64, 1)

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,1])
x_test_zoom = np.zeros([x_test.shape[0],size,size,1],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,1) /255

In [17]:
x_test_zoom.shape

(3588, 64, 64, 1)

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11(input_shape=(64,64,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [19]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(64, 64, 1), classes=7, include_top=True,pooling=None, weights = None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         80        
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32) 

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 56s 2ms/sample - loss: 1.8123 - acc: 0.2523 - macro_f1score: 1.9034e-04 - weighted_f1score: 3.5307e-05 - val_loss: 1.7858 - val_acc: 0.2502 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 55s 2ms/sample - loss: 1.7063 - acc: 0.3125 - macro_f1score: 0.0478 - weighted_f1score: 0.0096 - val_loss: 1.6093 - val_acc: 0.3789 - val_macro_f1score: 0.0835 - val_weighted_f1score: 0.0163
Epoch 3/100
28698/28698 [==============================] - 55s 2ms/sample - loss: 1.4984 - acc: 0.4146 - macro_f1score: 0.1598 - weighted_f1score: 0.0304 - val_loss: 1.4341 - val_acc: 0.4469 - val_macro_f1score: 0.1763 - val_weighted_f1score: 0.0330
Epoch 4/100
28698/28698 [==============================] - 55s 2ms/sample - loss: 1.3904 - acc: 0.4591 - macro_f1score: 0.2056 - weighted_f1score: 0.0385 - val_loss: 1.3656 - val_acc: 0

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 2s 595us/sample - loss: 2.2388 - acc: 0.5231 - macro_f1score: 0.4705 - weighted_f1score: 0.0729

Accuracy: 0.5231, Macro F1 Score: 0.4705, Weighted F1 Score: 0.0729


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [0]:
W

[array([[[[-0.20710374, -0.07064918, -0.0594946 ,  0.22597471,
           -0.04779955,  0.03186734,  0.26725504, -0.21047749]],
 
         [[-0.24466497,  0.2560319 ,  0.17534307, -0.1968757 ,
           -0.16281419, -0.00409541,  0.02781034,  0.15685634]],
 
         [[ 0.14758381, -0.15777074, -0.23045118,  0.21146534,
           -0.11525122,  0.33312467, -0.04984443, -0.12360132]]],
 
 
        [[[ 0.20141992, -0.11700282,  0.15035795,  0.00534261,
           -0.13224776, -0.06056001, -0.0650178 ,  0.04985527]],
 
         [[ 0.16558172,  0.08551399,  0.00672219,  0.13246013,
            0.10880127, -0.3288409 , -0.16499202,  0.28900278]],
 
         [[-0.05454913,  0.2926067 , -0.31735057,  0.0981431 ,
            0.18918219,  0.18142107,  0.01678084,  0.16887933]]],
 
 
        [[[-0.11833178, -0.11658107, -0.28030217,  0.2758732 ,
            0.3213977 ,  0.28084585,  0.19673063, -0.27016133]],
 
         [[ 0.2581601 , -0.22033203,  0.29711345,  0.09128069,
           -0.1424687

In [0]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1_1/kernel:0' shape=(3, 3, 1, 8) dtype=float32>,
 <tf.Variable 'block1_conv1_1/bias:0' shape=(8,) dtype=float32>,
 <tf.Variable 'block2_conv1_1/kernel:0' shape=(3, 3, 8, 16) dtype=float32>,
 <tf.Variable 'block2_conv1_1/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'block3_conv1_1/kernel:0' shape=(3, 3, 16, 32) dtype=float32>,
 <tf.Variable 'block3_conv1_1/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block3_conv2_1/kernel:0' shape=(3, 3, 32, 32) dtype=float32>,
 <tf.Variable 'block3_conv2_1/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block4_conv1_1/kernel:0' shape=(3, 3, 32, 64) dtype=float32>,
 <tf.Variable 'block4_conv1_1/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block4_conv2_1/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block4_conv2_1/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block5_conv1_1/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block5_conv1_1/bias:0' shape=(64,) dtype=float32>,
 <t

In [0]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG16(input_shape=(64,64,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(64, 64, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         80        
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 123s 4ms/sample - loss: 1.8148 - acc: 0.2487 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8165 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 123s 4ms/sample - loss: 1.8118 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8167 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 122s 4ms/sample - loss: 1.8121 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8165 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 122s 4ms/sample - loss: 1.8118 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 4s 1ms/sample - loss: 1.8138 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


### 3) My VGG19 (Pretrained)

In [0]:
# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.


def VGG19(input_shape=(64,64,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(64, 64, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         80        
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, 64, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 32, 32, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 140s 5ms/sample - loss: 1.8150 - acc: 0.2497 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8196 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 140s 5ms/sample - loss: 1.8127 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8157 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/100
28698/28698 [==============================] - 139s 5ms/sample - loss: 1.8119 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8157 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 4/100
28698/28698 [==============================] - 139s 5ms/sample - loss: 1.8117 - acc: 0.2514 - macro_f1score: 0.0000e+00 - weighted_f1score

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 4s 1ms/sample - loss: 1.8117 - acc: 0.2494 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00

Accuracy: 0.2494, Macro F1 Score: 0.0000, Weighted F1 Score: 0.0000


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.


# 1. 224의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. Conv의 filter는 1/8로 줄인다.
# 4. 다음과 같이 모형을 재구성한다.
# 5. 위의 내용은 앞으로 비교될 모형에서도 공통적으로 작용한다.

def VGG11_init(input_shape=(64,64,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(64, 64, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 64, 64, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, 64, 8)         80        
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 32, 32, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 32, 32, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 16, 16, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 16, 16, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 16, 16, 32) 

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [0]:
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 56s 2ms/sample - loss: 1.6895 - acc: 0.3152 - macro_f1score: 0.0589 - weighted_f1score: 0.0111 - val_loss: 1.5531 - val_acc: 0.3957 - val_macro_f1score: 0.1399 - val_weighted_f1score: 0.0289
Epoch 2/100
28698/28698 [==============================] - 57s 2ms/sample - loss: 1.4506 - acc: 0.4330 - macro_f1score: 0.1715 - weighted_f1score: 0.0328 - val_loss: 1.3731 - val_acc: 0.4667 - val_macro_f1score: 0.1978 - val_weighted_f1score: 0.0374
Epoch 3/100
28698/28698 [==============================] - 55s 2ms/sample - loss: 1.3216 - acc: 0.4884 - macro_f1score: 0.2371 - weighted_f1score: 0.0435 - val_loss: 1.2911 - val_acc: 0.5015 - val_macro_f1score: 0.2295 - val_weighted_f1score: 0.0428
Epoch 4/100
28698/28698 [==============================] - 55s 2ms/sample - loss: 1.2338 - acc: 0.5243 - macro_f1score: 0.2896 - weighted_f1score: 0.0519 - val_loss: 1.2586 - val_acc: 0.5099 - val_macr

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 2s 592us/sample - loss: 1.7255 - acc: 0.5588 - macro_f1score: 0.4950 - weighted_f1score: 0.0773

Accuracy: 0.5588, Macro F1 Score: 0.4950, Weighted F1 Score: 0.0773


## 3. For Size =48,
---

### 1) My VGG11 (Pretraining)

In [0]:
# 오리지널 데이터에 대해 다루어본다.

def VGG11(input_shape=(48,48,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu',name='fc1')(x)
        x = Dense(256, activation='relu', name='fc2')(x)
        output = Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
#내 데이터 맞춤형 모형
model = VGG11(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         80        
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32) 

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [0]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 35s 1ms/sample - loss: 1.7050 - acc: 0.3200 - macro_f1score: 0.0624 - weighted_f1score: 0.0124 - val_loss: 1.6387 - val_acc: 0.3488 - val_macro_f1score: 0.0961 - val_weighted_f1score: 0.0214
Epoch 2/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.5082 - acc: 0.4134 - macro_f1score: 0.1560 - weighted_f1score: 0.0298 - val_loss: 1.4508 - val_acc: 0.4383 - val_macro_f1score: 0.1838 - val_weighted_f1score: 0.0346
Epoch 3/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.4086 - acc: 0.4543 - macro_f1score: 0.1983 - weighted_f1score: 0.0371 - val_loss: 1.3892 - val_acc: 0.4695 - val_macro_f1score: 0.1851 - val_weighted_f1score: 0.0346
Epoch 4/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.3352 - acc: 0.4868 - macro_f1score: 0.2407 - weighted_f1score: 0.0438 - val_loss: 1.3457 - val_acc: 0.4845 - val_macr

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 1s 382us/sample - loss: 1.6702 - acc: 0.4869 - macro_f1score: 0.4095 - weighted_f1score: 0.0675

Accuracy: 0.4869, Macro F1 Score: 0.4095, Weighted F1 Score: 0.0675


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

In [0]:
W

[array([[[[ 7.00309640e-03, -2.40320906e-01, -1.24406725e-01,
           -2.29221322e-02,  1.25448713e-02,  1.39155775e-01,
            1.18913330e-01, -2.91309003e-02]],
 
         [[ 5.88647649e-02,  7.17299506e-02, -1.10253230e-01,
           -1.99645981e-01, -1.41429052e-01,  1.69789329e-01,
            2.55907536e-01, -9.78955030e-02]],
 
         [[ 1.70848668e-01, -5.36267124e-02,  2.02521801e-01,
           -1.75346017e-01,  6.47457410e-03, -1.62362024e-01,
            1.11016683e-01, -1.00963891e-01]]],
 
 
        [[[ 1.17977150e-01,  1.01499118e-01, -1.10974282e-01,
           -6.35014102e-02, -2.08860099e-01, -8.39651283e-03,
            2.42345273e-01, -2.73578316e-01]],
 
         [[ 1.78281188e-01, -2.62544274e-01,  1.64805427e-01,
            1.95294842e-01,  6.72122464e-02, -2.48201475e-01,
           -2.29138345e-01,  8.34240764e-02]],
 
         [[-4.91929846e-03, -2.45614033e-02, -1.03713892e-01,
            5.47641031e-02,  1.97933897e-01,  4.26596403e-02,
        

In [0]:
model.weights # 참고. get_weights()와 weights는 다른 매서드임을 알자.

[<tf.Variable 'block1_conv1_5/kernel:0' shape=(3, 3, 1, 8) dtype=float32>,
 <tf.Variable 'block1_conv1_5/bias:0' shape=(8,) dtype=float32>,
 <tf.Variable 'block2_conv1_5/kernel:0' shape=(3, 3, 8, 16) dtype=float32>,
 <tf.Variable 'block2_conv1_5/bias:0' shape=(16,) dtype=float32>,
 <tf.Variable 'block3_conv1_5/kernel:0' shape=(3, 3, 16, 32) dtype=float32>,
 <tf.Variable 'block3_conv1_5/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block3_conv2_5/kernel:0' shape=(3, 3, 32, 32) dtype=float32>,
 <tf.Variable 'block3_conv2_5/bias:0' shape=(32,) dtype=float32>,
 <tf.Variable 'block4_conv1_5/kernel:0' shape=(3, 3, 32, 64) dtype=float32>,
 <tf.Variable 'block4_conv1_5/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block4_conv2_5/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block4_conv2_5/bias:0' shape=(64,) dtype=float32>,
 <tf.Variable 'block5_conv1_5/kernel:0' shape=(3, 3, 64, 64) dtype=float32>,
 <tf.Variable 'block5_conv1_5/bias:0' shape=(64,) dtype=float32>,
 <t

In [0]:
# 형태는 이렇게 생김
np.array(W).shape

(22,)

### 2) My VGG16 (Pretrained)

In [0]:
def VGG16(input_shape=(48,48,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
model = VGG16(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         80        
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [0]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.7611 - acc: 0.2762 - macro_f1score: 0.0162 - weighted_f1score: 0.0024 - val_loss: 1.6603 - val_acc: 0.3461 - val_macro_f1score: 0.1378 - val_weighted_f1score: 0.0262
Epoch 2/100
28698/28698 [==============================] - 68s 2ms/sample - loss: 1.5341 - acc: 0.3943 - macro_f1score: 0.1439 - weighted_f1score: 0.0279 - val_loss: 1.4398 - val_acc: 0.4416 - val_macro_f1score: 0.1758 - val_weighted_f1score: 0.0334
Epoch 3/100
28698/28698 [==============================] - 68s 2ms/sample - loss: 1.4027 - acc: 0.4555 - macro_f1score: 0.1998 - weighted_f1score: 0.0373 - val_loss: 1.4080 - val_acc: 0.4511 - val_macro_f1score: 0.1699 - val_weighted_f1score: 0.0329
Epoch 4/100
28698/28698 [==============================] - 68s 2ms/sample - loss: 1.3236 - acc: 0.4919 - macro_f1score: 0.2368 - weighted_f1score: 0.0433 - val_loss: 1.3119 - val_acc: 0.4896 - val_macr

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 2s 673us/sample - loss: 1.6827 - acc: 0.5346 - macro_f1score: 0.4613 - weighted_f1score: 0.0730

Accuracy: 0.5346, Macro F1 Score: 0.4613, Weighted F1 Score: 0.0730


### 3) My VGG19 (Pretrained)

In [0]:
def VGG19(input_shape=(48,48,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv4')(x)
    x = MaxPooling2D((3, 3), strides=(3, 3), name='block5_pool')(x) 
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.   

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='vgg19')

    # Load weights.
    # 내 모형에서는 쓸모없다. 다만, 나중의 혹시모를 참고를 위해 코드는 남겨놓는다.
    if weights == 'imagenet':
        if include_top:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels.h5',
                WEIGHTS_PATH,cache_subdir='models',file_hash='cbe5617147190e668d6c5d5026f83318')
        else:
            weights_path = get_file('vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5',
                WEIGHTS_PATH_NO_TOP,cache_subdir='models',file_hash='253f8cb515780f3b799900260a226db6')

        model.load_weights(weights_path) #경로에 있는 초기치 weights가져오기

    elif weights is not None:
        model.load_weights(weights)

    return model



In [0]:
#내 데이터 맞춤형 모형
# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         80        
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 8)         584       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 16)        2320      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0     

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')


In [0]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 78s 3ms/sample - loss: 1.7612 - acc: 0.2704 - macro_f1score: 0.0171 - weighted_f1score: 0.0019 - val_loss: 1.6993 - val_acc: 0.3031 - val_macro_f1score: 0.0663 - val_weighted_f1score: 0.0078
Epoch 2/100
28698/28698 [==============================] - 78s 3ms/sample - loss: 1.5952 - acc: 0.3605 - macro_f1score: 0.1173 - weighted_f1score: 0.0207 - val_loss: 1.5161 - val_acc: 0.3973 - val_macro_f1score: 0.1481 - val_weighted_f1score: 0.0270
Epoch 3/100
28698/28698 [==============================] - 78s 3ms/sample - loss: 1.4752 - acc: 0.4197 - macro_f1score: 0.1649 - weighted_f1score: 0.0311 - val_loss: 1.4646 - val_acc: 0.4101 - val_macro_f1score: 0.1601 - val_weighted_f1score: 0.0318
Epoch 4/100
28698/28698 [==============================] - 79s 3ms/sample - loss: 1.3951 - acc: 0.4571 - macro_f1score: 0.1986 - weighted_f1score: 0.0371 - val_loss: 1.3815 - val_acc: 0.4687 - val_macr

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))


3588/3588 [==============================] - 3s 722us/sample - loss: 1.6319 - acc: 0.4997 - macro_f1score: 0.4199 - weighted_f1score: 0.0700

Accuracy: 0.4997, Macro F1 Score: 0.4199, Weighted F1 Score: 0.0700


### 4) My VGG11 (Convergence Speed Comparison)

In [0]:
# Vgg11에서, pretraining 된 fc층을 가지고 VGG 11을 재학습 시킬때, 얼마나 적은 epoch만에 수렴하는지를 보기 위해 만들었다.

def VGG11_init(input_shape=(48,48,1), classes=7,include_top=True,pooling=None, weights = None):

    img_input = Input(shape=input_shape)

    # Block 1
    x = Conv2D(8, (3, 3), activation='relu', padding='same', name='block1_conv1')(img_input)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(16, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)
    # 데이터 size = 48로 하므로써, 전체를 maxpooling 하는 것으로 바꿈.

    if include_top:
        # Classification block
        x = Flatten(name='flatten')(x)
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-6]), bias_initializer= tf.constant_initializer(W[-5]) , name='fc1')(x) # 초기값은     tf.constant_initializer     로 한다!!
        x = Dense(256, activation='relu', kernel_initializer= tf.constant_initializer(W[-4]), bias_initializer= tf.constant_initializer(W[-3]) , name='fc2')(x)
        output = Dense(classes, kernel_initializer=tf.constant_initializer(W[-2]) , bias_initializer= tf.constant_initializer(W[-1]) , activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            output = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            output = GlobalMaxPooling2D()(x)

    # Create model.
    model = Model(img_input, output, name='Vgg11(Pretraining)')

    if weights is not None:
        model.load_weights(weights)

    return model


In [0]:
model = VGG11_init(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
model.summary()

Model: "Vgg11(Pretraining)"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 48, 48, 1)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 8)         80        
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 8)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 16)        1168      
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 16)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 12, 12, 32)        4640      
_________________________________________________________________
block3_conv2 (Conv2D)        (None, 12, 12, 32) 

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

early_stopping = EarlyStopping(monitor = 'val_macro_f1score',patience = 3 , verbose=1,mode='max')

In [0]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 35s 1ms/sample - loss: 1.7641 - acc: 0.3021 - macro_f1score: 0.0386 - weighted_f1score: 0.0064 - val_loss: 1.6354 - val_acc: 0.3653 - val_macro_f1score: 0.0649 - val_weighted_f1score: 0.0110
Epoch 2/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.5376 - acc: 0.4047 - macro_f1score: 0.1424 - weighted_f1score: 0.0268 - val_loss: 1.5084 - val_acc: 0.4113 - val_macro_f1score: 0.1744 - val_weighted_f1score: 0.0319
Epoch 3/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.4469 - acc: 0.4425 - macro_f1score: 0.1832 - weighted_f1score: 0.0336 - val_loss: 1.4163 - val_acc: 0.4539 - val_macro_f1score: 0.1872 - val_weighted_f1score: 0.0340
Epoch 4/100
28698/28698 [==============================] - 34s 1ms/sample - loss: 1.3647 - acc: 0.4747 - macro_f1score: 0.2233 - weighted_f1score: 0.0404 - val_loss: 1.3786 - val_acc: 0.4689 - val_macr

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 387us/sample - loss: 2.2315 - acc: 0.4978 - macro_f1score: 0.4312 - weighted_f1score: 0.0703

Accuracy: 0.4978, Macro F1 Score: 0.4312, Weighted F1 Score: 0.0703


## 4. For Size = 48, No Early Stopping
---

### 1) Epoch = 50

#### (1) My VGG11

In [0]:
############# 이 아래는 early stopping 없이 진행 #######################

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [26]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 31s 1ms/sample - loss: 1.7759 - acc: 0.2735 - macro_f1score: 0.0177 - weighted_f1score: 0.0029 - val_loss: 1.6611 - val_acc: 0.3444 - val_macro_f1score: 0.0999 - val_weighted_f1score: 0.0199
Epoch 2/50
28698/28698 [==============================] - 31s 1ms/sample - loss: 1.5534 - acc: 0.3958 - macro_f1score: 0.1372 - weighted_f1score: 0.0264 - val_loss: 1.5384 - val_acc: 0.4196 - val_macro_f1score: 0.1493 - val_weighted_f1score: 0.0299
Epoch 3/50
28698/28698 [==============================] - 30s 1ms/sample - loss: 1.4452 - acc: 0.4439 - macro_f1score: 0.1826 - weighted_f1score: 0.0346 - val_loss: 1.4208 - val_acc: 0.4547 - val_macro_f1score: 0.1754 - val_weighted_f1score: 0.0328
Epoch 4/50
28698/28698 [==============================] - 31s 1ms/sample - loss: 1.3682 - acc: 0.4723 - macro_f1score: 0.2219 - weighted_f1score: 0.0409 - val_loss: 1.3724 - val_acc: 0.4617 - val_macro_f1

In [27]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 333us/sample - loss: 3.3225 - acc: 0.5109 - macro_f1score: 0.4549 - weighted_f1score: 0.0734

Accuracy: 0.5109, Macro F1 Score: 0.4549, Weighted F1 Score: 0.0734


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [31]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 61s 2ms/sample - loss: 1.7147 - acc: 0.3073 - macro_f1score: 0.0443 - weighted_f1score: 0.0096 - val_loss: 1.5472 - val_acc: 0.4060 - val_macro_f1score: 0.1365 - val_weighted_f1score: 0.0273
Epoch 2/50
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.4928 - acc: 0.4217 - macro_f1score: 0.1552 - weighted_f1score: 0.0302 - val_loss: 1.4482 - val_acc: 0.4377 - val_macro_f1score: 0.1993 - val_weighted_f1score: 0.0364
Epoch 3/50
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.4011 - acc: 0.4642 - macro_f1score: 0.2016 - weighted_f1score: 0.0378 - val_loss: 1.3879 - val_acc: 0.4636 - val_macro_f1score: 0.2193 - val_weighted_f1score: 0.0396
Epoch 4/50
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.3304 - acc: 0.4915 - macro_f1score: 0.2367 - weighted_f1score: 0.0435 - val_loss: 1.4099 - val_acc: 0.4450 - val_macro_f1

In [32]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 2s 634us/sample - loss: 3.1222 - acc: 0.5309 - macro_f1score: 0.4705 - weighted_f1score: 0.0755

Accuracy: 0.5309, Macro F1 Score: 0.4705, Weighted F1 Score: 0.0755


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [35]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.8032 - acc: 0.2524 - macro_f1score: 4.4168e-04 - weighted_f1score: 1.0112e-04 - val_loss: 1.7368 - val_acc: 0.2901 - val_macro_f1score: 7.0373e-04 - val_weighted_f1score: 7.1473e-05
Epoch 2/50
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.6530 - acc: 0.3307 - macro_f1score: 0.0780 - weighted_f1score: 0.0134 - val_loss: 1.5678 - val_acc: 0.3859 - val_macro_f1score: 0.1269 - val_weighted_f1score: 0.0218
Epoch 3/50
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.4733 - acc: 0.4258 - macro_f1score: 0.1659 - weighted_f1score: 0.0312 - val_loss: 1.4974 - val_acc: 0.4068 - val_macro_f1score: 0.1641 - val_weighted_f1score: 0.0306
Epoch 4/50
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.3730 - acc: 0.4691 - macro_f1score: 0.2067 - weighted_f1score: 0.0384 - val_loss: 1.3409 - val_acc: 0.479

In [36]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 2s 693us/sample - loss: 2.2415 - acc: 0.5449 - macro_f1score: 0.4883 - weighted_f1score: 0.0788

Accuracy: 0.5449, Macro F1 Score: 0.4883, Weighted F1 Score: 0.0788


### 2) Epoch = 100

#### (1) My VGG11

In [0]:
# VGG 11 - pretraining
model = VGG11(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [39]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 30s 1ms/sample - loss: 1.7202 - acc: 0.3103 - macro_f1score: 0.0545 - weighted_f1score: 0.0118 - val_loss: 1.5811 - val_acc: 0.3837 - val_macro_f1score: 0.1501 - val_weighted_f1score: 0.0288
Epoch 2/100
28698/28698 [==============================] - 30s 1ms/sample - loss: 1.5063 - acc: 0.4163 - macro_f1score: 0.1593 - weighted_f1score: 0.0303 - val_loss: 1.4576 - val_acc: 0.4391 - val_macro_f1score: 0.1587 - val_weighted_f1score: 0.0298
Epoch 3/100
28698/28698 [==============================] - 30s 1ms/sample - loss: 1.4044 - acc: 0.4618 - macro_f1score: 0.2003 - weighted_f1score: 0.0376 - val_loss: 1.4395 - val_acc: 0.4517 - val_macro_f1score: 0.2216 - val_weighted_f1score: 0.0422
Epoch 4/100
28698/28698 [==============================] - 30s 1ms/sample - loss: 1.3226 - acc: 0.4955 - macro_f1score: 0.2431 - weighted_f1score: 0.0445 - val_loss: 1.3495 - val_acc: 0.4778 - val_macr

In [40]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 1s 356us/sample - loss: 3.9843 - acc: 0.5020 - macro_f1score: 0.4409 - weighted_f1score: 0.0703

Accuracy: 0.5020, Macro F1 Score: 0.4409, Weighted F1 Score: 0.0703


In [0]:
# 한층당 W 와 b , 2개씩 있으므로 11개층이라면 22개의 모수 벡터 및 행렬이 출력된다.
W = model.get_weights()

#### (2) My VGG16

In [0]:
# VGG 16 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG16(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [44]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 61s 2ms/sample - loss: 1.7393 - acc: 0.2880 - macro_f1score: 0.0290 - weighted_f1score: 0.0041 - val_loss: 1.6197 - val_acc: 0.3664 - val_macro_f1score: 0.1223 - val_weighted_f1score: 0.0252
Epoch 2/100
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.5351 - acc: 0.3975 - macro_f1score: 0.1472 - weighted_f1score: 0.0277 - val_loss: 1.4597 - val_acc: 0.4255 - val_macro_f1score: 0.1574 - val_weighted_f1score: 0.0305
Epoch 3/100
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.4285 - acc: 0.4445 - macro_f1score: 0.1828 - weighted_f1score: 0.0344 - val_loss: 1.3980 - val_acc: 0.4634 - val_macro_f1score: 0.1692 - val_weighted_f1score: 0.0328
Epoch 4/100
28698/28698 [==============================] - 60s 2ms/sample - loss: 1.3584 - acc: 0.4747 - macro_f1score: 0.2206 - weighted_f1score: 0.0409 - val_loss: 1.3703 - val_acc: 0.4712 - val_macr

In [45]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 2s 635us/sample - loss: 3.4740 - acc: 0.5340 - macro_f1score: 0.4735 - weighted_f1score: 0.0765

Accuracy: 0.5340, Macro F1 Score: 0.4735, Weighted F1 Score: 0.0765


#### (3) My VGG19

In [0]:
# VGG 19 - pretrained

# 모수가 데이터에 비해 굉장히 많기 때문에 모수의 수를 좀 줄여서 확인해보자.
# 기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!

model = VGG19(input_shape=(48, 48, 1), classes=7, include_top=True,pooling=None, weights = None)


In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [48]:
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.7980 - acc: 0.2528 - macro_f1score: 9.0909e-04 - weighted_f1score: 9.7173e-05 - val_loss: 1.7854 - val_acc: 0.2513 - val_macro_f1score: 8.2102e-04 - val_weighted_f1score: 7.0556e-05
Epoch 2/100
28698/28698 [==============================] - 69s 2ms/sample - loss: 1.6890 - acc: 0.3073 - macro_f1score: 0.0521 - weighted_f1score: 0.0066 - val_loss: 1.6245 - val_acc: 0.3511 - val_macro_f1score: 0.0960 - val_weighted_f1score: 0.0162
Epoch 3/100
28698/28698 [==============================] - 68s 2ms/sample - loss: 1.5409 - acc: 0.3926 - macro_f1score: 0.1403 - weighted_f1score: 0.0265 - val_loss: 1.5019 - val_acc: 0.4101 - val_macro_f1score: 0.1287 - val_weighted_f1score: 0.0247
Epoch 4/100
28698/28698 [==============================] - 71s 2ms/sample - loss: 1.4281 - acc: 0.4419 - macro_f1score: 0.1793 - weighted_f1score: 0.0337 - val_loss: 1.3978 - val_acc: 0

In [49]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 3s 711us/sample - loss: 2.8764 - acc: 0.5362 - macro_f1score: 0.4760 - weighted_f1score: 0.0752

Accuracy: 0.5362, Macro F1 Score: 0.4760, Weighted F1 Score: 0.0752


## 5. Model Save & Import
---

In [0]:
#참고.
model.save('CNN model/VGG19.h5') # 모델 저장

In [0]:
model.save_weights('CNN model/VGG19_weights.h5') # 가중치 저장

In [0]:
model2 = load_model('CNN model/VGG19.h5',custom_objects={"macro_f1score": macro_f1score,"weighted_f1score":weighted_f1score}) # 모델 불러오기  custom_objects={"":} 로 정의했던 측도들 불러온다.